In [8]:
import pandas as pd


I will be using two more packages, pickle which allows to easily save data to disk and re which allows us to use regular expression syntax. Finally, for nicer table formatting, we import IPython.display.

In [9]:
import pickle
import re
from IPython.display import display, HTML


In [10]:
# defining a dictioanary below for Census data. Keys will be census years.

Census_raw = {}

In [11]:
for year in range(1790,2020,10):
    Census_raw[year]=pd.read_html('https://en.wikipedia.org/wiki/{}_United_States_Census'.format(year))

In [12]:
Census_raw[2019]= pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population', header = [1])
display(Census[2019].head(1))

NameError: name 'Census' is not defined

In [ ]:
if len(Census_raw.keys()):
    with open('rawdata.pickle','wb') as handle:
        pickle.dump(Census_raw, handle, protocol = pickle.HIGHEST_PROTOCOL)
else:
    with open('rawdata.pickle', 'rb') as handle:
        Census_raw = pickle.load(handle)

# Data Cleaning
All Census pages contain several tables but we are only interested in those tables that contain State (or District as in 1800) as one of their columns. Most pages also have a seperate statistic on largest cities and we thus ignore all tables that have City as one of their columns.




In [13]:

Census = {}
for key in Census_raw.keys():
    Census[key] = [table for table in Census_raw[key] if ('State' in table.columns) or (('District' in table.columns))]
    if len(Census[key]) > 1:
        Census[key] = [table for table in Census[key] if ('City' not in table.columns)][0]
    else:
        Census[key] = Census[key][0]


In [14]:
for key in Census.keys():
    print("Displaying values for {}".format(key))
    display(Census[key].head(1))

Displaying values for 1790


,State,"Free white males of 16 years and upward, including heads of families",Free white males under 16 years,"Free white females, including heads of families",All other free persons,Slaves,Enslaved % of state population,Total,% of US population
0,Vermont,22435,22328,40505,255,16[a][7],nil,"85,539[b]",2.2%


Displaying values for 1800


,District,"Free white males under age 10,",Free white males age 10–16,Free white males age 16–26,Free white males age 26–45,Free white males over age 45,Free white females under age 10,Free white females age 10–16,Free white females age 16–26,Free white females age 26–45,Free white females over age 45,All other free persons,Slaves,Total
0,New Hampshire,30694,14881,16379,17589,11715,29871,14193,17153,18381,12142,852,8,183858


Displaying values for 1810


...

,Rank,State,Population as of1990 Census[4],Population as of2000 Census[4],Change,Percentchange
0,1,California,29760021,33871648,4111627,13.8%


Displaying values for 2010


,Rank,State,Population as of2000 Census,Population as of2010 Census[51],Change,Percentchange
0,1,California,33871648,37253956,3382308,10.0%


Displaying values for 2019


,Current,2010,State,"Estimate, April 1, 2019[5]","April 1, 2010[6]",Percent[note 3],Absolute,Total U.S. House of Representatives Seats,"Estimated population per electoral vote, 2019[note 1]","Estimated, 2019",2010.1,"Percent of the total U.S. population, 2018[note 2]"
0,1.0,1.0,California,39512223,37254523,6.1%,"+2,257,700",53,718404,745514,702885,11.96%


In [15]:
Census[1790].rename(columns ={'Total':'Population'}, inplace = True)
Census[1800].rename(columns= {'Total': 'Population','District':'State'}, inplace = True)

In [16]:
for year in range (1990,2020,10):
    d = {'Population as of{}.*'.format(year): 'Population'}
    Census[year].columns = Census[year].columns.to_series().replace(d, regex= True)
    Census[2019].columns = Census[2019].columns.to_series().replace({'Estimate, April 1, 2019.*': 'Population'}, regex= True)     

In [17]:
 for key in Census.keys():
    print("Displaying values for {}".format(key))
    Census[key] = Census[key][['State','Population']]
    Census[key].loc[:,'Population']=Census[key].loc[:,'Population'].apply(
        #lambda s: int(re.sub(r'\]\[].*','', str(s)).replace(',','')))
        lambda s: int(re.sub('[^0-9]','', str(s))))
    display(Census[key].head(8))

Displaying values for 1790


/Users/nitinsharma/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,State,Population
0,Vermont,85539
1,New Hampshire,141885
2,Maine,96540
3,Massachusetts,3787878
4,Rhode Island,68825
5,Connecticut,237946
6,New York,340120
7,New Jersey,184139


Displaying values for 1800


,State,Population
0,New Hampshire,183858
1,Massachusetts,422845
2,Maine,151719
3,Connecticut,251002
4,Vermont,154465
5,Rhode Island,69122
6,"New York (excluding Duchess, Ulster, Orange co...",484065
7,"New York (Duchess, Ulster, Orange counties)",101985


...

,State,Population
0,California,33871648
1,Texas,20851820
2,New York,18976457
3,Florida,15982378
4,Illinois,12419293
5,Pennsylvania,12281054
6,Ohio,11353140
7,Michigan,9938444


Displaying values for 2010


,State,Population
0,California,37253956
1,Texas,25145561
2,New York,19378102
3,Florida,18801310
4,Illinois,12830632
5,Pennsylvania,12702379
6,Ohio,11536504
7,Michigan,9883640


Displaying values for 2019


,State,Population
0,California,39512223
1,Texas,28995881
2,Florida,21477737
3,New York,19453561
4,Pennsylvania,12801989
5,Illinois,12671821
6,Ohio,11689100
7,Georgia,10617423


In [18]:
 display(Census[1800].head(10))
#two rows for NY, values shoulbe be aggregated into 1 row

,State,Population
0,New Hampshire,183858
1,Massachusetts,422845
2,Maine,151719
3,Connecticut,251002
4,Vermont,154465
5,Rhode Island,69122
6,"New York (excluding Duchess, Ulster, Orange co...",484065
7,"New York (Duchess, Ulster, Orange counties)",101985
8,New Jersey,211149
9,Pennsylvania (eastern district),327979


In [27]:
Census[key].to_csv('States.csv')

States = pd.read_csv('States.csv')['State']
StatesList = States.tolist()

if isinstance(Census[1790].index[0], int):
    for key in Census.keys():
        for i, row in Census[key].iterrows():
            for State in StatesList:
                if State in row['State']:
                    Census[key].loc[i,'State'] = State  
        Census[key] = Census[key][Census[key]['State'].isin(StatesList)]
        Census[key] = Census[key].groupby(['State'],squeeze=True).sum()
        Census[key].rename(columns={'Population':str(key)},inplace=True)
display(Census[1800].head(14))

,1800
State,
Connecticut,251002
Delaware,64273
District of Columbia,323297
Georgia,162686
Indiana,5641
Kentucky,220959
Maine,151719
Maryland,4276
Massachusetts,422845


In [25]:
pd.read_csv('States.csv')

,Unnamed: 0,State,Population
0,0,California,39512223
1,1,Texas,28995881
2,2,Florida,21477737
3,3,New York,19453561
4,4,Pennsylvania,12801989
5,5,Illinois,12671821
6,6,Ohio,11689100
7,7,Georgia,10617423
8,8,North Carolina,10488084
9,9,Michigan,9986857


In [32]:

result_df = pd.DataFrame(States).set_index('State')
for key in Census.keys():
    result_df = result_df.add(Census[key],fill_value=0)
result_df.to_csv('State_Census_Historical.csv')
result_df.tail(5)

,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
State,,,,,,,,,,,,,,,,,,,,,
Virginia,7476108.0,880200.0,877683.0,938261.0,1044054.0,1025227.0,1119348.0,1596318.0,1225163.0,1542180.0,...,2421851.0,2677773.0,3262000.0,3978000.0,4648494.0,5346818.0,6187358.0,7078515.0,8001024.0,8535519.0
Washington,NaN,NaN,NaN,NaN,NaN,NaN,1201.0,11594.0,23955.0,98268.0,...,1563396.0,1736191.0,2386000.0,2860000.0,3409169.0,4132156.0,4866692.0,5894121.0,6724540.0,7614893.0
West Virginia,NaN,NaN,105469.0,136808.0,176924.0,224537.0,302313.0,NaN,442014.0,648936.0,...,1729205.0,1961974.0,2006000.0,1857000.0,1744237.0,1949644.0,1793477.0,1808344.0,1852994.0,1787147.0
Wisconsin,NaN,NaN,NaN,1444.0,3635.0,30945.0,305391.0,775881.0,1054670.0,1399750.0,...,2939006.0,3137587.0,3449000.0,3964000.0,4417933.0,4705767.0,4891769.0,5363675.0,5686986.0,5822434.0
Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9118.0,32610.0,...,225565.0,250742.0,292000.0,338000.0,332416.0,469557.0,453588.0,493782.0,563626.0,578759.0


In [33]:
#pd.read_csv('States.csv')
pd.read_csv('State_Census_Historical.csv')



,State,1790,1800,1810,1820,1830,1840,1850,1860,1870,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
0,Alabama,NaN,NaN,9046.0,127901.0,309527.0,590756.0,771623.0,964201.0,996992.0,...,2646248.0,2832961.0,3060000.0,3273000.0,3444165.0,3893888.0,4040587.0,4447100.0,4779736.0,4903185.0
1,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59278.0,72524.0,138000.0,200000.0,302173.0,401851.0,550043.0,626932.0,710231.0,731545.0
2,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55641.0
3,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9658.0,...,435573.0,499261.0,756000.0,1318000.0,1772482.0,2718215.0,3665228.0,5130632.0,6392017.0,7278717.0
4,Arkansas,NaN,NaN,1062.0,14273.0,30388.0,97574.0,209897.0,435450.0,484471.0,...,1854482.0,1949387.0,1906000.0,1788000.0,1923295.0,2286435.0,2350725.0,2673400.0,2915918.0,3017825.0
5,California,NaN,NaN,NaN,NaN,NaN,NaN,92597.0,379994.0,560247.0,...,5677251.0,6907387.0,10586223.0,2085000.0,19953134.0,23667902.0,29760021.0,33871648.0,37253956.0,39512223.0
6,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34277.0,39864.0,...,1035791.0,1123296.0,1337000.0,1758000.0,2207259.0,2889964.0,3294394.0,4301261.0,5029196.0,5758736.0
7,Connecticut,237946.0,251002.0,262042.0,275202.0,297675.0,309978.0,370792.0,460147.0,537454.0,...,1606903.0,1709242.0,2007280.0,2548000.0,3032217.0,3107576.0,3287116.0,3405565.0,3574097.0,3565287.0
8,Contiguous United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325386357.0
9,Delaware,59094.0,64273.0,72674.0,72749.0,76748.0,78085.0,91532.0,112216.0,125015.0,...,238380.0,266505.0,321000.0,449000.0,548104.0,594338.0,666168.0,783600.0,897934.0,973764.0


In [39]:
result_df.interpolate(axis = 1,inplace = True )
result_df.to_csv('State_Census_Historical.csv')
result_df.tail()

,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
State,,,,,,,,,,,,,,,,,,,,,
Virginia,7476108.0,880200.0,877683.0,938261.0,1044054.0,1025227.0,1119348.0,1596318.0,1225163.0,1542180.0,...,2421851.0,2677773.0,3262000.0,3978000.0,4648494.0,5346818.0,6187358.0,7078515.0,8001024.0,8535519.0
Washington,NaN,NaN,NaN,NaN,NaN,NaN,1201.0,11594.0,23955.0,98268.0,...,1563396.0,1736191.0,2386000.0,2860000.0,3409169.0,4132156.0,4866692.0,5894121.0,6724540.0,7614893.0
West Virginia,NaN,NaN,105469.0,136808.0,176924.0,224537.0,302313.0,372163.5,442014.0,648936.0,...,1729205.0,1961974.0,2006000.0,1857000.0,1744237.0,1949644.0,1793477.0,1808344.0,1852994.0,1787147.0
Wisconsin,NaN,NaN,NaN,1444.0,3635.0,30945.0,305391.0,775881.0,1054670.0,1399750.0,...,2939006.0,3137587.0,3449000.0,3964000.0,4417933.0,4705767.0,4891769.0,5363675.0,5686986.0,5822434.0
Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9118.0,32610.0,...,225565.0,250742.0,292000.0,338000.0,332416.0,469557.0,453588.0,493782.0,563626.0,578759.0


In [38]:
pd.read_csv('State_Census_Historical.csv')


,State,1790,1800,1810,1820,1830,1840,1850,1860,1870,...,1930,1940,1950,1960,1970,1980,1990,2000,2010,2019
0,Alabama,NaN,NaN,9046.0,127901.0,309527.0,590756.0,771623.0,964201.0,996992.0,...,2646248.0,2832961.0,3060000.0,3273000.0,3444165.0,3893888.0,4040587.0,4447100.0,4779736.0,4903185.0
1,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59278.0,72524.0,138000.0,200000.0,302173.0,401851.0,550043.0,626932.0,710231.0,731545.0
2,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55641.0
3,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9658.0,...,435573.0,499261.0,756000.0,1318000.0,1772482.0,2718215.0,3665228.0,5130632.0,6392017.0,7278717.0
4,Arkansas,NaN,NaN,1062.0,14273.0,30388.0,97574.0,209897.0,435450.0,484471.0,...,1854482.0,1949387.0,1906000.0,1788000.0,1923295.0,2286435.0,2350725.0,2673400.0,2915918.0,3017825.0
5,California,NaN,NaN,NaN,NaN,NaN,NaN,92597.0,379994.0,560247.0,...,5677251.0,6907387.0,10586223.0,2085000.0,19953134.0,23667902.0,29760021.0,33871648.0,37253956.0,39512223.0
6,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34277.0,39864.0,...,1035791.0,1123296.0,1337000.0,1758000.0,2207259.0,2889964.0,3294394.0,4301261.0,5029196.0,5758736.0
7,Connecticut,237946.0,251002.0,262042.0,275202.0,297675.0,309978.0,370792.0,460147.0,537454.0,...,1606903.0,1709242.0,2007280.0,2548000.0,3032217.0,3107576.0,3287116.0,3405565.0,3574097.0,3565287.0
8,Contiguous United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,325386357.0
9,Delaware,59094.0,64273.0,72674.0,72749.0,76748.0,78085.0,91532.0,112216.0,125015.0,...,238380.0,266505.0,321000.0,449000.0,548104.0,594338.0,666168.0,783600.0,897934.0,973764.0


## See Visualization of this data https://public.flourish.studio/visualisation/1488228/
